#### For every routine
#### Sample times per activity
#### Check for overlaps and get actions in order
#### Create single script with times
#### Execute to get timestamped graphs ***

In [ ]:
%load_ext autoreload
%autoreload 2

import json
import os
import shutil
import numpy as np
from random import random
import sys

sys.path.append('simulation')
from evolving_graph.scripts import Script, parse_script_line
import evolving_graph.utils as utils
from evolving_graph.scripts import read_script, read_script_from_string, read_script_from_list_string, ScriptParseException, Action
from evolving_graph.execution import ScriptExecutor
from evolving_graph.environment import EnvironmentGraph

from GraphReader import GraphReader, init_graph_file, scene_num
from ProgramExecutor import read_program

In [ ]:
def time_internal(mins, hrs, day=0):
    return int(round(((((day)*24)+hrs)*60+mins)/DT))

def time_human(time_int):
    time_mins = time_int * DT
    mins = time_mins%60
    time_mins = time_mins//60
    hrs = time_mins%24
    time_mins = time_mins//24
    days = time_mins
    h = '{:02d}:{:02d}'.format(hrs,mins)
    if days != 0:
        h = str(days)+'day - '+h
    return h

def day_num(day_of_week):
    return {'Monday':0,'Tuesday':1,'Wednesday':2,'Thursday':3,'Friday':4,'Saturday':5,'Sunday':6}[day_of_week]

In [ ]:
init_graph = GraphReader(graph_file=init_graph_file)
print(f'Using scene {int(scene_num)-1}, i.e. \'TestScene{scene_num}\'')

In [ ]:
DT = 1   # minutes
NUM_ROUTINES = 20
WEEKEND_DAYS = [day_num(day) for day in ['Saturday','Sunday']]
START_TIME = time_internal(mins=0, hrs=6)
INTERLEAVING = True

ignore_classes = ['floor','wall','ceiling','character']

In [ ]:
class Activity():
    def __init__(self, name, time, script_file=None):
        self.name = name
        start_time = time_internal(time[0][1], time[0][0])
        end_time = time_internal(time[1][1], time[1][0])
        directory = os.path.join('sourcedScripts', name)
        if script_file is None:
            script_file = np.random.choice(os.listdir(directory))
            headers, self.scripts, _ = read_program(os.path.join(directory,script_file), init_graph.node_map)
        def sample_duration(header):
            durations = (header).split('-')
            assert len(durations)==2, f"Invalid comment {header} in {name}->{script_file}"
            duration_min = int(durations[0].strip())
            duration_max = int(durations[1].strip())
            duration_sampled = (random() * (duration_max-duration_min) + duration_min)/DT
            return duration_sampled
        self.durations = [sample_duration(header) for header in headers]
        def valid_times(times):
            for time, next_time, duration in zip(times[:-1], times[1:], self.durations[:-1]):
                if next_time - time < duration:
                    return False
            return True
        valid = False
        while not valid:
            times = (np.random.rand(len(headers)) * (end_time-start_time) + start_time).round().astype(int)
            times.sort()
            valid = valid_times(times)
        self.times = times
    
    def get_action_info(self):
        detailed_actions = []
        for t,d,scr in zip(self.times, self.durations, self.scripts):
            t2 = int(round(t+d))
            detailed_actions.append({'time_from':t, 'time_to':t2, 'name':self.name, 'script':scr, 'time_from_h':time_human(t), 'time_to_h':time_human(t2)})
        return detailed_actions

class Schedule():
    def __init__(self, type=None):
        assert type == None or type in ['weekday','weekend']
        schedule_options = []
        for (root,_,files) in os.walk('sourcedSchedules'):
            if type is not None and type not in root:
                continue
            schedule_options += [os.path.join(root,f) for f in files]
        schedule_file_path = np.random.choice(schedule_options)
        with open(schedule_file_path) as f:
            schedule = json.load(f)
        self.activities = [Activity(act_name, act_time) for act_name,act_time in schedule.items()]
    
    def get_combined_script(self, verbose=False):
        all_actions = []
        for act in self.activities:
            all_actions += act.get_action_info()
        all_actions.sort(key = lambda x : x['time_from'])
        if verbose:
            for a in all_actions:
                print (a['time_from_h']+' to '+a['time_to_h']+' : '+a['name'])
                for l in a['script']:
                    print(' - ',l)
        valid=True
        for end_time, next_start in zip([a['time_to'] for a in all_actions[:-1]], [a['time_from'] for a in all_actions[1:]]):
            if end_time>next_start:
                valid = False
                if verbose:
                    print(f'End time {end_time} of an activity, exceeds start time {next_start} of next activity')
                break
        return all_actions, valid

def get_graphs(all_actions):
    with open (init_graph_file,'r') as f:
        init_graph_dict = json.load(f)
    name_equivalence = utils.load_name_equivalence()
    graphs = [EnvironmentGraph(init_graph_dict).to_dict()]
    for action in all_actions:
        print('## Executing '+action['name']+' from '+action['time_from_h']+' to '+action['time_to_h'])
        executor = ScriptExecutor(EnvironmentGraph(graphs[-1]), name_equivalence)
        # for l in action['script']:
        #     print (l)
        success, state, _ = executor.execute(Script(action['script']), w_graph_list=False)
        graphs.append(state.to_dict())
        # print_graph_difference(graphs[-2],graphs[-1])
        # input('Press something...')
        if not success:
            print('Execution of {} starting at {} failed because {}'.format(action['name'], action['time_from_h'], executor.info.get_error_string()))
    return graphs, success



In [ ]:
def remove_ignored_classes(graphs):
    clipped_graphs = []
    for graph in graphs:
        clipped_graphs.append({'nodes':[],'edges':[]})
        ignore_ids = []
        for node in graph['nodes']:
            if node['class_name'] in ignore_classes:
                ignore_ids.append(node['id'])
            else:
                clipped_graphs[-1]['nodes'].append(node)
        for edge in graph['edges']:
            if edge['from_id'] in ignore_ids or edge['to_id'] in ignore_ids:
                continue
            clipped_graphs[-1]['edges'].append(edge)
    return clipped_graphs

def class_from_id(graph, id):
    lis = [n['class_name'] for n in graph['nodes'] if n['id']==id]
    if len(lis) > 0:
        return lis[0]
    else:
        return 'None'

def print_graph_difference(g1,g2):
    edges_removed = [e for e in g1['edges'] if e not in g2['edges']]
    edges_added = [e for e in g2['edges'] if e not in g1['edges']]
    nodes_removed = [n for n in g1['nodes'] if n['id'] not in [n2['id'] for n2 in g2['nodes']]]
    nodes_added = [n for n in g2['nodes'] if n['id'] not in [n2['id'] for n2 in g1['nodes']]]

    for n in nodes_removed:
        print ('Removed node : ',n)
    for n in nodes_added:
        print ('Added node   : ',n)
    for e in edges_removed:
        c1 = class_from_id(g1,e['from_id'])
        c2 = class_from_id(g1,e['to_id'])
        if c1 != 'character' and c2 != 'character' and e['relation_type'] in ['INSIDE','ON']:
            print ('Removed edge : ',c1,e['relation_type'],c2)
    for e in edges_added:
        c1 = class_from_id(g2,e['from_id'])
        c2 = class_from_id(g2,e['to_id'])
        if c1 != 'character' and c2 != 'character' and e['relation_type'] in ['INSIDE','ON']:
            print ('Added edge   : ',c1,e['relation_type'],c2)

In [ ]:
data = []
failed_routines = 0
successful_routines = 0

DATASET_DIR = 'sourcedRoutines/trial'
if os.path.exists(DATASET_DIR):
    shutil.rmtree(DATASET_DIR)
os.makedirs(DATASET_DIR)

while successful_routines < NUM_ROUTINES:
    day = np.random.choice(['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'])
    day_number = day_num(day)
    day_type = 'weekend' if day_number in WEEKEND_DAYS else 'weekday'
    s = Schedule(day_type)
    actions, valid = s.get_combined_script()
    if not valid:
        failed_routines += 1
        continue
    graphs, success = get_graphs(actions)
    graphs = remove_ignored_classes(graphs)
    if not success:
        failed_routines += 1
        continue
    times = [START_TIME] + [a['time_to'] for a in actions]
    data.append({'times':times,'graphs':graphs})
    successful_routines += 1

with open(os.path.join(DATASET_DIR,'sample.json'), 'w') as f:
    json.dump(data, f)


In [ ]:
edge_classes = ["INSIDE", "ON", "CLOSE"]
nodes = data[0]['graphs'][0]['nodes']
with open(os.path.join(DATASET_DIR,'classes.json'), 'w') as f:
    json.dump({"nodes":nodes, "edges":edge_classes, "dt":DT}, f)

In [ ]:
info = {}
info['dt'] = DT
info['num_routines'] = NUM_ROUTINES
info['weekend_days'] = WEEKEND_DAYS
info['start_time'] = START_TIME
info['interleaving'] = INTERLEAVING
info['num_nodes'] = len(nodes)
print(info)
with open(os.path.join(DATASET_DIR,'info.json'), 'w') as f:
    json.dump(info, f)